In [ ]:
model.load_state_dict(torch.load("best_model.pth"))


In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device, threshold = 0.5):
    # if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
    #     raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")
    best_val_loss = float('inf')
    best_model_state = None
    wait = 0
    n_count = 0
    criterion.to(device) #? Move criterion to device
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            n_count += inputs.size(0) #? Count number of samples trained
                                    
            # print(all(param.requires_grad for param in model.parameters()))
   
            torch.set_printoptions(threshold=float('inf'))
            assert not torch.isnan(inputs).any(), "Input has NaNs"
            assert not torch.isinf(inputs).any(), "Input has Infs"
            outputs = model(inputs) #? Forward pass through the model
            assert not torch.isnan(outputs).any(), "Model output has NaNs"
            assert not torch.isinf(outputs).any(), "Model output has Infs"
            loss = criterion(outputs, labels) #? Calculate loss
            assert not torch.isnan(loss).any(), "Model loss has NaNs"
            loss.backward() #? Backpropagation
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            running_loss += loss.item()
            optimiser.step() #? Update weights
            # scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        # print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
            avg_val_loss = val_loss / len(val_loader)
            loss_ratio = val_loss / train_loss    
            pos_weight = loss_ratio  # or any other function of loss_ratio you choose
            # scheduler.step(val_loss)  # Use validation loss here, not training loss
            scheduler.step()

        # Update criterion with new pos_weight
        # criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))
        # Early stopping
        if avg_val_loss < best_val_loss and epoch > 100:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        elif avg_val_loss*0.96 <= best_val_loss:
                wait = 0
        else:
            wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch}, best val loss: {best_val_loss:.4f}")
            break
        print(f"Epoch: {epoch}".ljust(12), 
              f"training loss:{train_loss:.3f}".ljust(16), 
              f"best_val_loss:{best_val_loss:.3f}".ljust(12), 
              f"Val Loss: {avg_val_loss:.3f}",
              f"Scheduler lr: {scheduler.get_last_lr()}".ljust(50), 
              f"N samples trained: {n_count}",
              end="\r")
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                # predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                predictions = (torch.sigmoid(outputs) >= threshold).float().cpu().numpy()
                
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy=accuracy_score(labels, predictions) 
    precision=precision_score(labels, predictions, pos_label=1, zero_division=0)
    recall=recall_score(labels, predictions, pos_label=1)
    f1=f1_score(labels, predictions, pos_label=1)
    auc=roc_auc_score(labels, predictions)
    
    return model, accuracy, precision, recall, f1, auc

In [ ]:
from Training_Helper_Functions import *
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import numpy as np
import optuna
from torch import optim
def maximise_combined_score(trial):
    if True:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", device)
        epochs = 800
        random_state = 42
        # Scaler
        scaler = RobustScaler() # Or StandardScaler() - You can also make this a hyperparameter if you want
        threshold = trial.suggest_float("threshold", 0.28,0.6,)
        # dropout = None
        initial_lr = trial.suggest_float("initial_lr", 1e-3, 1e-3 ,log=True)
        max_lr = trial.suggest_float("max_lr", 1e-3, 1e-3, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 6e-1, log=True)
        # Loss function hyperparameters
        criterion_choice = "BCEWithLogitsLoss" #trial.suggest_categorical("criterion", ["BCEWithLogitsLoss", "FocalLoss"])
        # Hyperparameter exploration optimization
        if criterion_choice == "BCEWithLogitsLoss":
            pos_weight = trial.suggest_int("pos_weight",3, 3)
            alpha = None
            gamma = None
        elif criterion_choice == "FocalLoss":
            pos_weight = None
            alpha = trial.suggest_float("alpha", 0.25, 0.75)
            gamma = trial.suggest_float("gamma", 1.0, 5.0)
        else:
            pos_weight = None

        # Initialize lists for metrics across folds
        accuracy_list = []
        precision_list = []
        recall_list = []
        f1_list = []
        auc_list = []
    if True:
        training_set = pd.read_csv('./DATA/folds/train_fold_0.csv')
        cv_set = pd.read_csv('./DATA/folds/test_fold_0.csv')
        holdout_set = pd.read_csv('./DATA/holdout_set/holdout_data_OHE.csv')
        # print(training_set.shape)
        # print(cv_set.shape)
        # print(holdout_set.shape)

        train_x = training_set.drop(columns = 'DR')
        train_y = training_set[['DR']]
        test_x = cv_set.drop(columns = 'DR')
        test_y = cv_set[['DR']] 
    # Cross-validation loop
    # for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
    train_x, test_x, train_y, test_y
    fold = 1
    print(f"Fold {fold}:")
    # Create DataLoader for current fold
    train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=512,
                                                        device=device)
    # Instantiate and initialize the model
    model = Ivan_NN(input_dim=get_feature_count(train_loader))
    model.to(device)
    model.apply(init_weights)

    # Map the choice to the actual loss function
    criterion = criterion_mapping(criterion_choice, pos_weight, alpha, gamma)
    optimiser = optim.Adam(model.parameters(), lr=initial_lr, weight_decay=weight_decay) 
    scheduler = torch.optim.lr_scheduler.CyclicLR(
        optimiser,
        base_lr=1e-6,
        max_lr=max_lr,
        cycle_momentum=True)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min', patience=3, factor=0.5)

    # scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=10, gamma=0.1)

    # Train and evaluate the model on the current fold
    model, accuracy, precision, recall, f1, auc = train_and_evaluate(
        model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=epochs, patience=40,
        device=device, threshold=threshold
    )
    print(f"Accuracy: {accuracy:.4f}, precision: {precision:.4f}, recall: {recall:.4f}, f1: {f1:.4f}, auc: {auc:.4f}")
    del model
    del train_loader
    del val_loader

    # Append the metrics from the current fold
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    # combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5
    combined_score = avg_f1

    return combined_score